# TEACh TfD language model dataset generation

The "make_dataset" function processes TEACh traj_data of the chosen split (train, valid_seen, valid_unseen) and outputs the NL instructions of the queried type (film, no_recept) with the corresponding lists of subtasks.


There are two functions: *get_actions* and *get_template_actions*. The first outputs the GT trajectories in 'no_recept' ALFRED style. The second implements manually designed templates for each TEACh task type.

In [ ]:
import json
import string
import numpy as np 
import os
import pickle

%cd ../..

In [4]:
from alfred_utils.gen.constants import *

exclude = set(string.punctuation)

PATH_TO_SCENE_NAMES = 'teach_utils/data/tfd_instances/'
            
def make_dataset(split: str, instr_type: str):
    
    frames = []
    path = PATH_TO_SCENE_NAMES + split
    games = list(filter(lambda x: '.json' in x, os.listdir(path=path)))
    for game in games:
        with open(path + '/' + game, 'r') as f:
            ep = json.load(f)
        if instr_type == 'film':
            actions = get_template_actions(ep, path)
        elif instr_type == 'no_recept':
            actions = get_actions(ep, path)
        else:
            print('Unknown instructions type')
            return

        frame = {}
        frame['code'] = ' ; '.join(' '.join((t[0], t[1])) for t in actions)
        frame['nl'] = ' '.join(': '.join(utt) for utt in ep['dialog_history_cleaned'])
        frame['list_of_actions'] = actions
        frame['game_id'] = ep['game_id']
        frames.append(frame)
        
    return frames


def get_actions(ep: dict, path: str):
    """
    A function for ground-truth trajectories generation. 
    For 'no_recept' instructions type.
    """
    actions = []
    for act_dict in ep['driver_actions_future']:
        oid = act_dict['oid']
        if oid:
            obj_tokens = oid.split('|')
            obj = obj_tokens[0] if 'Sliced' not in obj_tokens[-1] else obj_tokens[-1][:-2]
            if obj in ('Sink', 'Bathtub'):
                obj += 'Basin'
            action = act_dict['action_name']
            if action in ('Pickup', 'Open', 'Close', 'Slice'):
                action += 'Object'
            elif action == 'ToggleOn':
                action = 'ToggleObjectOn'
            elif action == 'ToggleOff':
                action = 'ToggleObjectOff'
            elif action == 'Place':
                action = 'PutObject'
            elif action == 'Pour':
                action = 'EmptyLiquidFromObject'
            actions.append((obj, action))
    
    return actions


def get_template_actions(ep: dict, path: str):
    """
    A function that implements the templates for each TEACh task type.
    For 'film' instructions type.
    """
    task_type = ep['game']['tasks'][0]['task_name']
    components = ep['game']['tasks'][0]['components']

    # For each task we have a function that returns a sequence of subtasks.
    # Some tasks can be compound, i.e. some functions may call the others 
    # within
 
    def boil():
        # Boil potato in the pot
        actions = []
        actions.append(('Potato', 'PickupObject'))
        actions.append(('Pot', 'PutObject'))
        actions.append(('Pot', 'PickupObject'))
        actions.append(('StoveBurner', 'PutObject'))
        actions.append(('StoveBurner', 'ToggleObjectOn'))
        actions.append(('StoveBurner', 'ToggleObjectOff'))

        return actions
    
    def coffee(obj):
        # Clean the mug (cup) and make a coffee
        actions = []
        actions.append((obj, 'PickupObject')) 
        actions.append(('SinkBasin', 'PutObject')) 
        actions.append(('Faucet', 'ToggleObjectOn'))
        actions.append(('Faucet', 'ToggleObjectOff')) 
        actions.append((obj, 'PickupObject')) 
        actions.append((obj, 'EmptyLiquidFromObject')) 
        actions.append(('CoffeMachine', 'PutObject')) 
        actions.append(('CoffeMachine', 'ToggleObjectOn')) 
        actions.append(('CoffeeMachine', 'ToggleObjectOff'))

        return actions

    def water_plant(obj):
        # Water plant with the "obj" container
        actions = []
        actions.append((obj, 'PickupObject'))
        actions.append(('HousePlant', 'EmptyLiquidFromObject'))

        return actions

    def n_slices(obj, recept, n):
        # Slice "obj" and put "n" slices in "recept"
        actions = []
        actions.append(('Knife', 'PickupObject'))
        actions.append((obj, 'SliceObject'))
        actions.append(('CounterTop', 'PutObject'))
        for i in range(n):
            actions.append((obj + 'Sliced', 'PickupObject'))
            actions.append((recept, 'PutObject'))

        return actions
    
    def n_cooked_slices(obj, recept, n):
        # Cook "obj" in microwave, slice it and put "n" slices in "recept"
        actions = []
        actions.append((obj, 'PickupObject'))
        actions.append(('Microwave', 'OpenObject'))
        actions.append(('Microwave', 'PutObject'))
        actions.append(('Microwave', 'CloseObject'))  
        actions.append(('Microwave', 'ToggleObjectOn'))
        actions.append(('Microwave', 'ToggleObjectOff'))
        actions.append(('Microwave', 'OpenObject'))
        actions += n_slices(obj, recept, n)

        return actions
    
    def toast():
        # Cook the bread in toaster and put it on the plate
        actions = []        
        actions.append(('Knife', 'PickupObject'))
        actions.append(('Bread', 'SliceObject'))
        actions.append(('CounterTop', 'PutObject'))
        actions.append(('BreadSliced', 'PickupObject'))
        actions.append(('Toaster', 'PutObject'))
        actions.append(('Toaster', 'ToggleObjectOn'))
        actions.append(('Toaster', 'ToggleObjectOff'))
        actions.append(('BreadSliced', 'PickupObject'))
        actions.append(('Plate', 'PutObject'))
        
        return actions
    
    def salad():
        actions = []
        # Cook potato in the microwave
        actions.append(('Potato', 'PickupObject'))
        actions.append(('Microwave', 'OpenObject'))
        actions.append(('Microwave', 'PutObject'))
        actions.append(('Microwave', 'CloseObject'))  
        actions.append(('Microwave', 'ToggleObjectOn'))
        actions.append(('Microwave', 'ToggleObjectOff'))
        actions.append(('Microwave', 'OpenObject'))

        # Slice all the ingredients
        actions.append(('Knife', 'PickupObject'))
        actions.append(('Potato', 'SliceObject'))
        actions.append(('Tomato', 'SliceObject'))
        actions.append(('Lettuce', 'SliceObject'))
        actions.append(('CounterTop', 'PutObject'))

        # Put all the ingredients into the plate
        actions.append(('PotatoSliced', 'PickupObject'))
        actions.append(('Plate', 'PutObject'))
        actions.append(('TomatoSliced', 'PickupObject'))
        actions.append(('Plate', 'PutObject'))
        actions.append(('LettuceSliced', 'PickupObject'))
        actions.append(('Plate', 'PutObject'))

        return actions
    
    def sandwich():
        actions = []

        # Slice all the ingredients
        actions.append(('Knife', 'PickupObject'))
        actions.append(('Bread', 'SliceObject'))
        actions.append(('Tomato', 'SliceObject'))
        actions.append(('Lettuce', 'SliceObject'))
        actions.append(('CounterTop', 'PutObject'))

        # Make 2 toasts
        for i in range(2):
            actions.append(('BreadSliced', 'PickupObject'))
            actions.append(('Toaster', 'PutObject'))
            actions.append(('Toaster', 'ToggleObjectOn'))
            actions.append(('Toaster', 'ToggleObjectOff'))
            actions.append(('BreadSliced', 'PickupObject'))
            actions.append(('Plate', 'PutObject'))
        
        # Put the rest of the ingredients on the plate
        actions.append(('TomatoSliced', 'PickupObject'))
        actions.append(('Plate', 'PutObject'))
        actions.append(('LettuceSliced', 'PickupObject'))
        actions.append(('Plate', 'PutObject'))

        return actions
    
    # Call the corresponding function for each task type with the parameters
    if task_type == 'Boil X':
        actions = boil()

    elif task_type == 'Coffee':
        obj = components['coffee']['conditions']['objectType']
        actions = coffee(obj)

    elif task_type == 'Water Plant':
        container = 'Mug'
        # All the objects used in GT trajectory
        gt_objs = [obj for obj, _ in get_actions(ep, path)]
        # Search for scene objects that are filled with liquid and
        # present in GT trajectory
        for obj in ep['game']['tasks'][0]['episodes'][0]['initial_state']['objects']:
            if obj['isFilledWithLiquid'] and obj['name'].split('_')[0] in gt_objs:
                container = obj['name'].split('_')[0] 

        actions = water_plant(container)

    elif task_type == 'N Slices Of X In Y':
        for key in components.keys():
            if key in VAL_ACTION_OBJECTS['Sliceable']:
                obj = key
            else:
                recept = key
        n = int(components[obj]['determiner'])
        actions = n_slices(obj, recept, n)

    elif task_type == 'N Cooked Slices Of X In Y':
        for key in components.keys():
            if key in VAL_ACTION_OBJECTS['Sliceable']:
                obj = key
            else:
                recept = key
        n = int(components[obj]['determiner'])
        actions = n_cooked_slices(obj, recept, n)

    elif task_type == 'Plate Of Toast':
        actions = toast()

    elif task_type == 'Salad':
        actions = salad()

    elif task_type == 'Sandwich':
        actions = sandwich()

    elif task_type == 'Breakfast':
        # This task is compound, it uses the functions of other tasks
        # dict_keys(['coffee', 'toast', 'potatoes', 'apple', 'sandwich', 'salad', 'serving_spot'])
        actions = []
        serving_spot = components['serving_spot']['conditions']['objectType']
        for key in components.keys():
            detr = components[key]['determiner']
            if detr != '0':
                if key == 'coffee':
                    obj = components[key]['task']['components']['coffee']['conditions']['objectType']
                    actions += coffee(obj)
                    actions.append((obj, 'PickupObject'))
                    actions.append((serving_spot, 'PutObject'))
                elif key == 'toast':
                    actions += toast()
                    actions.append(('Plate', 'PickupObject'))
                    actions.append((serving_spot, 'PutObject'))
                elif key == 'potatoes':
                    obj = 'Potato'
                    recept = components[key]['task_params'][-1]
                    n = int(components[key]['task_params'][0])
                    actions += n_cooked_slices(obj, recept, n)
                    actions.append((recept, 'PickupObject'))
                    actions.append((serving_spot, 'PutObject'))
                elif key == 'apple':
                    obj = 'Apple'
                    recept = components[key]['task_params'][-1]
                    n = int(components[key]['task_params'][0])
                    actions += n_cooked_slices(obj, recept, n)
                    actions.append((recept, 'PickupObject'))
                    actions.append((serving_spot, 'PutObject'))
                elif key == 'sandwich':
                    actions += sandwich()
                    actions.append(('Plate', 'PickupObject'))
                    actions.append((serving_spot, 'PutObject'))
                elif key == 'salad':
                    actions += salad()
                    actions.append(('Plate', 'PickupObject'))
                    actions.append((serving_spot, 'PutObject'))
                else:
                    pass                 

    else:
        # For task types with 'All' determiner we don't have templates
        # and use the GT trajectories instead
        actions = get_actions(ep, path)
    
    return actions

In [5]:
split = 'valid_seen' # can also take the value of 'train', 'valid_unseen'
instr_type = 'no_recept' # can also take the value of 'film'
frames = make_dataset(split, instr_type)

In [6]:
frames[10]['list_of_actions']

[('Faucet', 'ToggleObjectOff'),
 ('Cabinet', 'OpenObject'),
 ('Cabinet', 'CloseObject'),
 ('Knife', 'PickupObject'),
 ('Lettuce', 'SliceObject'),
 ('Tomato', 'SliceObject'),
 ('DiningTable', 'PutObject'),
 ('Plate', 'PickupObject'),
 ('Pot', 'PutObject'),
 ('Pot', 'PickupObject'),
 ('Pot', 'PutObject'),
 ('Faucet', 'ToggleObjectOn'),
 ('Faucet', 'ToggleObjectOff'),
 ('Pot', 'PickupObject'),
 ('SinkBasin', 'EmptyLiquidFromObject'),
 ('SinkBasin', 'PutObject'),
 ('Plate', 'PickupObject'),
 ('DiningTable', 'PutObject'),
 ('LettuceSliced', 'PickupObject'),
 ('Plate', 'PutObject'),
 ('TomatoSliced', 'PickupObject'),
 ('Plate', 'PutObject'),
 ('TomatoSliced', 'PickupObject'),
 ('Plate', 'PutObject')]

### Create training sets for [CodeT5 training](https://github.com/salesforce/codet5)

The function below is used to create json files for CodeT5 training with special names (train.json, dev.json, test.json).

In [8]:
OUTPUT_PATH = 'fiqa/language_processing/processed_instructions/'

def make_codet5_training_set(frames: dict, split: str):
    
    # Shuffle the tasks
    shuffler = np.random.permutation(len(frames))
    frames = np.array(frames)[shuffler]

    # Create a file with the proper name for CodeT5 training,
    # validation and testing 
    if split == 'train':
        file_name = 'train'
    elif split == 'valid_seen':
        file_name = 'dev'
    elif split == 'valid_unseen':
        file_name = 'test'
    with open(OUTPUT_PATH + f'{file_name}.json', 'w') as f:
        # We need only natural language text and the sequence of subtasks (code)
        for frame in frames:
            new_frame = {}
            new_frame['code'] = frame['code']
            new_frame['nl'] = frame['nl']
            json_data = json.dumps(new_frame)
            f.write(json_data + '\n')

Run this multiple times with different splits to create train, dev and test sets for CodeT5 training.

In [9]:
make_codet5_training_set(frames, split)

### Create files with GT trajectories for FIQA oracle agent

In [10]:
def make_gt_trajectories(frames: dict, split: str, instr_type: str):
    new_frames = {}
    for frame in frames:
        task_key = frame['game_id']
        new_frames[task_key] = frame['list_of_actions']
    with open(
       OUTPUT_PATH + f'{split}_{instr_type}_gt_teach.p', 'wb') as f:
     pickle.dump(new_frames, f)

Run this multiple times with different splits to create GT trajectories for train, valid_seen and unseen. These files has to be processed by the lp_outputs.py script inside FIQA to obtain the GT instructions with the navigation inserted.

In [12]:
make_gt_trajectories(frames, split, instr_type)